In [31]:
import pickle
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
from surprise import KNNBasic
from surprise.model_selection import train_test_split
from surprise.model_selection import GridSearchCV
from surprise import Reader, Dataset
from surprise.model_selection import cross_validate
import surprise.accuracy

# Import Data User, Anime, UserAnime

In [32]:
folder = "../Dataset SKRIPSI/"

filename = folder + "users_data"
UsersDF = pickle.load(open(filename,'rb'))

filename = folder + "animes_data"
AnimesDF = pickle.load(open(filename,'rb'))

filename = folder + "scores_data"
ScoresDF = pickle.load(open(filename,'rb'))

# Membobotkan Nilai Status

In [33]:
conditions = [
    (ScoresDF['my_status'] == 2),
    (ScoresDF['my_status'] == 3),
    (ScoresDF['my_status'] == 6),
    (ScoresDF['my_status'] == 1),
    (ScoresDF['my_status'] == 4)
]

values = [5,2,2,3,1]

ScoresDF['status_bobot'] = np.select(conditions,values)

# Mengkalikan Rating dengan Status

In [34]:
ScoresDF['scoreXstatus'] = ScoresDF['my_score'] * ScoresDF['status_bobot']
ScoresDF[['scoreXstatus']] = MinMaxScaler().fit_transform(ScoresDF[['scoreXstatus']])
ScoresDF

,username,anime_id,my_score,my_status,animes_rated,status_bobot,scoreXstatus
0,Tsundora,21,2,4,259,1,0.020408
1,Tsundora,59,5,2,259,5,0.489796
2,Tsundora,210,5,2,259,5,0.489796
3,Tsundora,249,7,4,259,1,0.122449
4,Tsundora,269,6,2,259,5,0.591837
...,...,...,...,...,...,...,...
193267,Senpai_Helix,4214,10,2,4,5,1.000000
193268,Senpai_Helix,18677,9,2,4,5,0.897959
193269,Kiyoshi-Kun,1722,10,2,3,5,1.000000
193270,Kiyoshi-Kun,2993,10,2,3,5,1.000000


# Training dan Test Model
Training dan test model dipisah 80% : 20%

In [35]:
reader = Reader(rating_scale=(0,1))
data = Dataset.load_from_df(ScoresDF[['username','anime_id','scoreXstatus']],reader)

trainset, testset = train_test_split(data, test_size=0.2, random_state=50)

In [40]:
sim_options = {'name': 'pearson',
                'min_support': 1
               }
algo_knn = KNNBasic(min_k = 3,k=50, sim_options=sim_options)
prediction_knn = algo_knn.fit(trainset).test(testset)

# Prediksi
prediction_knn

Computing the pearson similarity matrix...
Done computing similarity matrix.


[Prediction(uid='Shouichirou', iid=12679, r_ui=0.7959183673469387, est=0.7145890741192206, details={'actual_k': 32, 'was_impossible': False}),
 Prediction(uid='aleigh', iid=2904, r_ui=0.8979591836734693, est=0.9336316064861726, details={'actual_k': 50, 'was_impossible': False}),
 Prediction(uid='bigkfc2k8', iid=267, r_ui=0.5918367346938774, est=0.7648050279628941, details={'actual_k': 50, 'was_impossible': False}),
 Prediction(uid='Torah', iid=6956, r_ui=1.0, est=0.7615586554644133, details={'actual_k': 50, 'was_impossible': False}),
 Prediction(uid='godT', iid=63, r_ui=0.693877551020408, est=0.6491139815811415, details={'actual_k': 50, 'was_impossible': False}),
 Prediction(uid='lightofzeref', iid=36511, r_ui=0.4081632653061224, est=0.38985079021809793, details={'actual_k': 14, 'was_impossible': False}),
 Prediction(uid='ranmaSA', iid=401, r_ui=1.0, est=0.8244946047737992, details={'actual_k': 33, 'was_impossible': False}),
 Prediction(uid='Josay', iid=16664, r_ui=0.8979591836734693, 

In [46]:
sim_matrix = algo_knn.compute_similarities()
predict = algo_knn.n_x
predict

1063

numpy.ndarray

# Mendapatkan List Rekomendasi

In [7]:
username = "ranmaSA"

# get the list of the movie ids
unique_ids = ScoresDF['anime_id'].unique()

# get the list of the ids that the username "user" has rated
iids = ScoresDF.loc[ScoresDF['username'] == username, 'anime_id']

# remove the rated anime for the recommendations
anime_to_predict = np.setdiff1d(unique_ids,iids)

In [8]:
user_recs = []
for iid in anime_to_predict:
    user_recs.append((iid, algo_knn.predict(uid='ranmaSA', iid=iid).est))

user_recommendations = pd.DataFrame(user_recs, columns=['iid', 'predict']).sort_values('predict', ascending=False)

# Nilai MAE dan RMSE

In [41]:
surprise.accuracy.mae(prediction_knn)
surprise.accuracy.rmse(prediction_knn)

MAE:  0.1506
RMSE: 0.2020


0.20201022058986942